# 课时53 利用回调函数使用Tensorboard等

In [10]:
import pandas as pd
import numpy as np
import seaborn as sb
sb.set_style('darkgrid')
# pathlib相比os.path更好用
import pathlib
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import datetime
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.4.0


In [2]:
# 加载MNIST数据集
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# 以tf.Dataset形式加载数据(对于MNIST数据集，它是没有第三个维度Channel的，这里添加上它的第三个维度)
train_images = tf.expand_dims(train_images, -1)
# 由于tf.GradientTape()要求的数据类型是float，因此这里需要改变MNIST数据集图片的数据类型
# 并在转换数据类型的过程中对图片数据进行归一化
train_images = tf.cast(train_images/255, tf.float32)
# 对于MNIST数据集的标签，数据类型是uint8(无符号8位)，这里为了方便计算，将其转换为int64类型
train_labels = tf.cast(train_labels, tf.int64)
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

test_images = tf.expand_dims(test_images, -1)
test_images = tf.cast(test_images/255, tf.float32)
test_labels = tf.cast(test_labels, tf.int64)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [3]:
# 对数据进行打乱以及batch划分
dataset = dataset.shuffle(buffer_size=60000).repeat().batch(batch_size=32)
test_dataset = test_dataset.repeat().batch(batch_size=32)
test_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [4]:
# 建立模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=[3, 3], activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=10, activation='softmax')])

In [5]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [6]:
# 定义Tensorboard的回调函数
import os
log_dir = os.path.join('logs', datetime.datetime.now().strftime('%Y%M%D-%H%M%S'))
# log_dir = os.path.join('logs')
# histogram_freq代表变量变化的直方图记录频次
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,                                                                     histogram_freq=1)

In [7]:
# 我们将学习率衰减的规则逻辑写成一个函数
def learning_rate_sche(epoch):
    learning_rate = 0.2
    if epoch > 2:
        learning_rate = 0.02
    if epoch > 10:
        learning_rate = 0.01
    if epoch > 20:
        learning_rate = 0.005
    # tf.summary.scalar用于记录某个标量值得变化
    tf.summary.scalar('learning_rate', data=learning_rate, step=epoch)
    return learning_rate

In [8]:
# 将控制学习率的逻辑写成一个函数，然后将这个函数作为参数传入到callbacks.LearningRateScheduler()中
# callbacks.LearningRateScheduler()会调用这个控制学习率的函数来改变学习率的速率
learning_rate_callback = tf.keras.callbacks.LearningRateScheduler(learning_rate_sche)

# 如果我们除了要按照逻辑对学习率进行改变，并且想要把学习率的变化情况给记录下来到磁盘上，则有下面：
# 创建一个文件编写器file_write
file_write = tf.summary.create_file_writer(logdir=os.path.join('logs/learning_rate/'))
# 设置file_write为默认的文件编写器, 当我们tf.summary.scalar记录完毕某个标量值的变化后，
# 它都会调用这个设置为默认的文件编写器的file_write把这个标量值得变化过程给写入磁盘
file_write.set_as_default()

In [9]:
model.fit(dataset, 
          epochs=3, 
          steps_per_epoch=60000//32, 
          validation_data=test_dataset,
          validation_steps=10000//32,
          callbacks=[tensorboard_callback, learning_rate_callback])
# 在上面的函数中传入learning_rate_callback就能按照我们设定的逻辑对学习率进行改变了

Epoch 1/3
1875/1875 [==============================] - 87s 46ms/step - loss: 2.4417 - accuracy: 0.1005 - val_loss: 2.3208 - val_accuracy: 0.1010
Epoch 2/3
1875/1875 [==============================] - 93s 50ms/step - loss: 2.3266 - accuracy: 0.1020 - val_loss: 2.3206 - val_accuracy: 0.0975
Epoch 3/3
1875/1875 [==============================] - 83s 44ms/step - loss: 2.3267 - accuracy: 0.1027 - val_loss: 2.3170 - val_accuracy: 0.1010


# 1. 在Notebook中加载Tensorboard

In [11]:
%load_ext tensorboard
%matplotlib inline

In [12]:
%tensorboard --logdir 章节8 Tensorboard可视化/logs/train

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--purge_orphaned_data BOOL] [--db URI] [--db_import]
                   [--inspect] [--version_tb] [--tag TAG] [--event_file PATH]
                   [--path_prefix PATH] [--window_title TEXT]
                   [--max_reload_threads COUNT] [--reload_interval SECONDS]
                   [--reload_task TYPE] [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [--whatif-use-unsafe-custom-prediction YOUR_CUSTOM_PREDICT_FUNCTION.py]
                   {serve,dev} ...
tensorboard: error: invalid choice: 'Tensorboard可视化/logs/train' (choose from 'serve', 'dev')

In [ ]:
# 在终端就直接执行下面的语句，但是需要注意的是，下面的语句中有中文，这是不对的，不能中文目录中
tensorboard --logdir 章节8 Tensorboard可视化/logs/train